# 🤖 VideoMAE - Reconocimiento de Lengua de Señas (WLASL)

**Proyecto**: Reconocimiento de Lengua de Señas Americana con VideoMAE

**Autor**: Rafael Ovalle - Tesis UNAB

**Modelo**: VideoMAE (Video Masked Autoencoder) de Microsoft Research

---

## 📋 Configuraciones Disponibles

| Config | Dataset | Train | Val | Test | Batch | LR | Regularización | Uso |
|--------|---------|-------|-----|------|-------|----|--------------|---------|
| **V1-100** | 100 clases | 807 | 194 | 117 | 16 | 1e-4 | Alta | Baseline |
| **V2-100** | 100 clases | 1,001 | 117 | 117 | 6 | 1e-5 | Baja | Maximizar datos |
| **V1-300** | 300 clases | 1,959 | 557 | 271 | 16 | 1e-4 | Alta | Baseline |
| **V2-300** | 300 clases | 2,516 | 271 | 271 | 6 | 1e-5 | Baja | Maximizar datos |

**Diferencias V1 vs V2:**
- **V1**: Split tradicional (train/val/test separados) + regularización completa
- **V2**: Train+Val combinados, Test como validación + regularización reducida

---

## 📚 Índice
1. [Configuración Inicial](#1)
2. [Verificar y Copiar Datasets](#2)
3. [Configurar Experimento](#3)
4. [Cargar Datos](#4)
5. [Entrenamiento](#5)
6. [Evaluación](#6)
7. [Visualizaciones](#7)
8. [Guardar y Descargar Resultados](#8)
9. [Utilidades](#9)
10. [Solución de Problemas](#10)

---
## 1️⃣ Configuración Inicial <a id="1"></a>

### Verificar GPU y Entorno

In [ ]:
# Verificar información del sistema
import sys
import torch

print("="*70)
print("INFORMACIÓN DEL SISTEMA".center(70))
print("="*70)
print(f"Python:  {sys.version.split()[0]}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA:    {torch.version.cuda if torch.cuda.is_available() else 'No disponible'}")
print(f"\nGPU disponible: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    gpu_memory_gb = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"Memoria GPU: {gpu_memory_gb:.2f} GB")
else:
    print("\n⚠️ WARNING: GPU no disponible!")
    print("   El entrenamiento será MUY lento.")
    print("   Solución: Runtime > Change runtime type > GPU")

print("="*70)

In [ ]:
# Verificar GPU con nvidia-smi
!nvidia-smi

### Montar Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Ruta donde guardas tus datasets y resultados en Drive
DRIVE_ROOT = "/content/drive/MyDrive/TESIS_WLASL"
print(f"✅ Drive montado: {DRIVE_ROOT}")

### Clonar Repositorio

In [ ]:
import os

# Clonar repositorio si no existe
if not os.path.exists('AtiendeSenas-MVP'):
    print("[INFO] Clonando repositorio...")
    !git clone https://github.com/Ov4llezz/AtiendeSenas-MVP.git
    %cd AtiendeSenas-MVP
else:
    print("[INFO] Repositorio ya existe, actualizando...")
    %cd AtiendeSenas-MVP
    !git pull

print("\n✅ Repositorio listo")
!pwd

### Instalar Dependencias

In [ ]:
# Instalar todas las dependencias necesarias
from colab_utils.config import setup_environment

print("[INFO] Instalando dependencias...\n")
setup_environment()
print("\n✅ Todas las dependencias instaladas")

---
## 2️⃣ Verificar y Copiar Datasets <a id="2"></a>

### Copiar Datasets desde Google Drive (Opcional)

Si tienes los datasets en Google Drive, cópialos al entorno de Colab para entrenamiento más rápido.

In [ ]:
import shutil
import os

# ============================================================
# CONFIGURA LAS RUTAS DE TUS DATASETS EN DRIVE
# ============================================================
DRIVE_DATASETS = "/content/drive/MyDrive/WLASL"  # Cambia esto a tu ruta
# ============================================================

# Rutas origen
src_wlasl100 = os.path.join(DRIVE_DATASETS, "wlasl100")
src_wlasl300 = os.path.join(DRIVE_DATASETS, "wlasl300")

# Ruta destino
dest_base = "/content/AtiendeSenas-MVP/data"
os.makedirs(dest_base, exist_ok=True)

def copiar_dataset(src, dest_base, dataset_name):
    """Copia dataset desde Drive a Colab"""
    if not os.path.exists(src):
        print(f"❌ No se encontró: {src}")
        return False
    
    destino = os.path.join(dest_base, dataset_name)
    
    # Si ya existe, preguntar si sobrescribir
    if os.path.exists(destino):
        print(f"⚠️  {dataset_name} ya existe en Colab, saltando...")
        return True
    
    print(f"📦 Copiando {dataset_name}... (esto puede tomar varios minutos)")
    shutil.copytree(src, destino)
    print(f"✅ {dataset_name} copiado exitosamente")
    return True

# Copiar datasets
print("\n" + "="*70)
print("COPIAR DATASETS DESDE DRIVE".center(70))
print("="*70 + "\n")

copiar_dataset(src_wlasl100, dest_base, "wlasl100")
copiar_dataset(src_wlasl300, dest_base, "wlasl300")

print("\n✅ Proceso completado")

### Verificar Estructura de Datasets

In [ ]:
import os

def verificar_dataset(dataset_path, dataset_name):
    """Verifica estructura y cuenta videos en cada split"""
    print(f"\n{'='*60}")
    print(f"DATASET: {dataset_name}".center(60))
    print("="*60)
    
    if not os.path.exists(dataset_path):
        print(f"❌ No encontrado: {dataset_path}")
        return
    
    splits = ['train', 'val', 'test']
    total_videos = 0
    
    for split in splits:
        split_path = os.path.join(dataset_path, 'dataset', split)
        
        if os.path.exists(split_path):
            videos = [f for f in os.listdir(split_path) if f.endswith('.mp4')]
            num_videos = len(videos)
            total_videos += num_videos
            print(f"{split.capitalize():5s}: {num_videos:5d} videos")
        else:
            print(f"{split.capitalize():5s}: ❌ No existe")
    
    print(f"{'─'*60}")
    print(f"TOTAL: {total_videos:5d} videos")
    print("="*60)

# Verificar ambos datasets
verificar_dataset('data/wlasl100', 'WLASL100')
verificar_dataset('data/wlasl300', 'WLASL300')

# Verificar versiones V2 si existen
if os.path.exists('data/wlasl100_v2'):
    verificar_dataset('data/wlasl100_v2', 'WLASL100_V2')
if os.path.exists('data/wlasl300_v2'):
    verificar_dataset('data/wlasl300_v2', 'WLASL300_V2')

---
## 3️⃣ Configurar Experimento <a id="3"></a>

### 🎯 Selecciona tu configuración de entrenamiento

In [ ]:
from colab_utils.config import create_config, print_config, save_config

# ============================================================
#   🎯 CONFIGURA TU EXPERIMENTO AQUÍ
# ============================================================

DATASET_TYPE = "wlasl100"  # Opciones: "wlasl100" o "wlasl300"
VERSION = "v1"             # Opciones: "v1" (baseline) o "v2" (experimental)

# ============================================================

# Crear configuración automática
config = create_config(
    dataset_type=DATASET_TYPE,
    version=VERSION,
    drive_root=DRIVE_ROOT
)

# Mostrar configuración
print_config(config)

# Guardar configuración (se guardará en results/)
save_config(config, config['results_dir'])

print(f"\n✅ Configuración creada: {DATASET_TYPE.upper()} {VERSION.upper()}")

### ⚙️ Ajustes Manuales (Opcional)

Si quieres personalizar hiperparámetros específicos:

In [ ]:
# Descomenta y modifica los parámetros que quieras cambiar

# config['batch_size'] = 8          # Cambiar batch size
# config['lr'] = 1e-4               # Cambiar learning rate
# config['max_epochs'] = 50         # Cambiar número de epochs
# config['patience'] = 10           # Cambiar paciencia early stopping
# config['num_workers'] = 2         # Cambiar workers DataLoader

print("Configuración actual:")
print(f"  Batch size:  {config['batch_size']}")
print(f"  Learning rate: {config['lr']}")
print(f"  Max epochs:  {config['max_epochs']}")
print(f"  Patience:    {config['patience']}")

---
## 4️⃣ Cargar Datos <a id="4"></a>

### Crear Datasets y DataLoaders

In [ ]:
from torch.utils.data import DataLoader
from colab_utils.dataset import WLASLVideoDataset

print("\n" + "="*70)
print("CARGANDO DATASETS".center(70))
print("="*70 + "\n")

# Crear datasets
print("[1/3] Creando Train Dataset...")
train_dataset = WLASLVideoDataset(
    split="train",
    base_path=config['data_root'],
    dataset_size=config['num_classes']
)

print("[2/3] Creando Validation Dataset...")
val_dataset = WLASLVideoDataset(
    split="val",
    base_path=config['data_root'],
    dataset_size=config['num_classes']
)

print("[3/3] Creando Test Dataset...")
test_dataset = WLASLVideoDataset(
    split="test",
    base_path=config['data_root'],
    dataset_size=config['num_classes']
)

# Crear dataloaders
print("\n[INFO] Creando DataLoaders...")
train_loader = DataLoader(
    train_dataset,
    batch_size=config['batch_size'],
    shuffle=True,
    num_workers=config['num_workers'],
    pin_memory=True if config['device'] == "cuda" else False
)

val_loader = DataLoader(
    val_dataset,
    batch_size=config['batch_size'],
    shuffle=False,
    num_workers=config['num_workers'],
    pin_memory=True if config['device'] == "cuda" else False
)

test_loader = DataLoader(
    test_dataset,
    batch_size=config['batch_size'],
    shuffle=False,
    num_workers=config['num_workers'],
    pin_memory=True if config['device'] == "cuda" else False
)

# Resumen
print("\n" + "="*70)
print("DATASETS CARGADOS EXITOSAMENTE".center(70))
print("="*70)
print(f"Train:      {len(train_dataset):>5,} videos ({len(train_loader):>4} batches)")
print(f"Validation: {len(val_dataset):>5,} videos ({len(val_loader):>4} batches)")
print(f"Test:       {len(test_dataset):>5,} videos ({len(test_loader):>4} batches)")
print(f"\nClases:     {config['num_classes']}")
print(f"Batch size: {config['batch_size']}")
print("="*70 + "\n")

---
## 5️⃣ Entrenamiento <a id="5"></a>

### Iniciar Entrenamiento Completo

⏰ **Tiempo estimado:**
- WLASL100 V1: ~2-3 horas (GPU T4)
- WLASL100 V2: ~3-4 horas (GPU T4)
- WLASL300 V1: ~6-8 horas (GPU T4)
- WLASL300 V2: ~8-12 horas (GPU T4)

In [ ]:
from colab_utils.training import train_model

print("\n" + "="*70)
print("🚀 INICIANDO ENTRENAMIENTO".center(70))
print("="*70 + "\n")

# Entrenar modelo
model, training_history, run_checkpoint_dir, log_dir = train_model(
    config=config,
    train_loader=train_loader,
    val_loader=val_loader,
    train_dataset=train_dataset
)

print("\n" + "="*70)
print("✅ ENTRENAMIENTO COMPLETADO".center(70))
print("="*70)
print(f"Checkpoints: {run_checkpoint_dir}")
print(f"Logs:        {log_dir}")
print("="*70 + "\n")

### 🎯 Entrenamiento Rápido (Solo para Pruebas)

Si solo quieres verificar que todo funciona correctamente:

In [ ]:
# SOLO DESCOMENTA SI QUIERES PRUEBA RÁPIDA (1-2 epochs)

# config_test = config.copy()
# config_test['max_epochs'] = 2
# config_test['patience'] = 5

# print("⚡ ENTRENAMIENTO DE PRUEBA (2 epochs)\n")
# model, training_history, run_checkpoint_dir, log_dir = train_model(
#     config=config_test,
#     train_loader=train_loader,
#     val_loader=val_loader,
#     train_dataset=train_dataset
# )

### Visualizar TensorBoard

Monitorea el entrenamiento en tiempo real:

In [ ]:
# Cargar extensión TensorBoard
%load_ext tensorboard

# Lanzar TensorBoard
%tensorboard --logdir {log_dir}

---
## 6️⃣ Evaluación <a id="6"></a>

### Cargar Mejor Modelo y Evaluar en Test Set

In [ ]:
from colab_utils.evaluation import evaluate_detailed, print_results, print_top_classes
import torch

print("\n" + "="*70)
print("CARGANDO MEJOR MODELO".center(70))
print("="*70 + "\n")

# Cargar mejor modelo
best_model_path = f"{run_checkpoint_dir}/best_model.pt"
checkpoint = torch.load(best_model_path, map_location=config['device'])
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"✅ Modelo cargado exitosamente")
print(f"   Epoch:    {checkpoint['epoch']}")
print(f"   Val Loss: {checkpoint['val_loss']:.4f}")
print(f"   Val Acc:  {checkpoint['val_acc']:.2f}%")

# Evaluar en test set
print("\n" + "="*70)
print("EVALUACIÓN EN TEST SET".center(70))
print("="*70 + "\n")

test_results = evaluate_detailed(
    model=model,
    dataloader=test_loader,
    device=config['device'],
    num_classes=config['num_classes']
)

# Mostrar resultados generales
print_results(test_results)

# Mostrar mejores y peores clases
print_top_classes(test_results, top_n=10)

---
## 7️⃣ Visualizaciones <a id="7"></a>

### Curvas de Entrenamiento

In [ ]:
import pandas as pd
from colab_utils.visualization import plot_training_curves
from datetime import datetime

# Preparar datos
history_df = pd.DataFrame(training_history)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Graficar curvas de entrenamiento
curves_path = f"{config['results_dir']}/training_curves_{timestamp}.png"
plot_training_curves(history_df, save_path=curves_path)

### Todas las Visualizaciones

Genera todas las visualizaciones: matriz de confusión, performance por clase, distribuciones, etc.

In [ ]:
from colab_utils.visualization import visualize_all_results

print("\n" + "="*70)
print("GENERANDO VISUALIZACIONES".center(70))
print("="*70 + "\n")

# Generar todas las visualizaciones
viz_paths = visualize_all_results(
    results=test_results,
    history_df=history_df,
    output_dir=config['results_dir'],
    timestamp=timestamp
)

print("\n" + "="*70)
print("✅ VISUALIZACIONES GENERADAS".center(70))
print("="*70)
for name, path in viz_paths.items():
    print(f"  {name:25s}: {path}")
print("="*70 + "\n")

---
## 8️⃣ Guardar y Descargar Resultados <a id="8"></a>

### Guardar Resultados Completos

In [ ]:
from colab_utils.evaluation import save_results

# Preparar información del checkpoint
checkpoint_info = {
    'best_epoch': int(checkpoint['epoch']),
    'best_val_loss': float(checkpoint['val_loss']),
    'best_val_acc': float(checkpoint['val_acc']),
    'total_epochs_trained': len(training_history),
}

# Guardar todos los resultados
json_path, txt_path, pred_path, ts = save_results(
    results=test_results,
    config=config,
    checkpoint_info=checkpoint_info,
    output_dir=config['results_dir']
)

print("\n" + "="*80)
print("📁 ARCHIVOS GENERADOS".center(80))
print("="*80)
print(f"Checkpoints:      {run_checkpoint_dir}")
print(f"Logs TensorBoard: {log_dir}")
print(f"JSON completo:    {json_path}")
print(f"Reporte TXT:      {txt_path}")
print(f"Predicciones CSV: {pred_path}")
print(f"Visualizaciones:  {config['results_dir']}")
print("="*80 + "\n")

### Copiar Resultados a Google Drive

In [ ]:
import shutil
import os

# Crear carpetas en Drive si no existen
drive_results = os.path.join(DRIVE_ROOT, "results")
drive_checkpoints = os.path.join(DRIVE_ROOT, "checkpoints")
drive_logs = os.path.join(DRIVE_ROOT, "logs")

os.makedirs(drive_results, exist_ok=True)
os.makedirs(drive_checkpoints, exist_ok=True)
os.makedirs(drive_logs, exist_ok=True)

print("\n📦 Copiando resultados a Google Drive...\n")

# Copiar resultados
dest_results = os.path.join(drive_results, os.path.basename(config['results_dir']))
if os.path.exists(dest_results):
    shutil.rmtree(dest_results)
shutil.copytree(config['results_dir'], dest_results)
print(f"✅ Resultados:   {dest_results}")

# Copiar checkpoints
dest_checkpoint = os.path.join(drive_checkpoints, os.path.basename(run_checkpoint_dir))
if os.path.exists(dest_checkpoint):
    shutil.rmtree(dest_checkpoint)
shutil.copytree(run_checkpoint_dir, dest_checkpoint)
print(f"✅ Checkpoints:  {dest_checkpoint}")

# Copiar logs
dest_logs = os.path.join(drive_logs, os.path.basename(log_dir))
if os.path.exists(dest_logs):
    shutil.rmtree(dest_logs)
shutil.copytree(log_dir, dest_logs)
print(f"✅ Logs:         {dest_logs}")

print("\n✅ Todos los archivos copiados a Google Drive")

### Descargar Resultados Comprimidos

In [ ]:
from google.colab import files
import os

# Crear nombre del archivo ZIP
experiment_name = f"{DATASET_TYPE}_{VERSION}_{timestamp}"
zip_name = f"results_{experiment_name}.zip"

# Comprimir todo
print(f"\n📦 Comprimiendo resultados...\n")
!zip -r -q {zip_name} \
    {config['results_dir']} \
    {run_checkpoint_dir} \
    {log_dir}

# Mostrar tamaño
zip_size_mb = os.path.getsize(zip_name) / (1024**2)
print(f"✅ Archivo creado: {zip_name} ({zip_size_mb:.2f} MB)")

# Descargar
print(f"\n⬇️  Descargando...")
files.download(zip_name)

print("\n✅ ¡Descarga completada!")

---
## 9️⃣ Utilidades <a id="9"></a>

### Monitorear Uso de GPU

In [ ]:
!nvidia-smi

### Limpiar Memoria GPU

In [ ]:
import gc
import torch

# Limpiar cache de GPU
gc.collect()
torch.cuda.empty_cache()

print("✅ Memoria GPU liberada")

# Verificar memoria disponible
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated() / 1024**3
    reserved = torch.cuda.memory_reserved() / 1024**3
    print(f"\nMemoria GPU:")
    print(f"  Allocated: {allocated:.2f} GB")
    print(f"  Reserved:  {reserved:.2f} GB")

### Ver Historial de Entrenamiento

In [ ]:
import pandas as pd

# Mostrar tabla de historial
history_df = pd.DataFrame(training_history)
print("\n📊 HISTORIAL DE ENTRENAMIENTO\n")
print(history_df.to_string(index=False))

# Mejores métricas
best_train_acc = history_df['train_acc'].max()
best_val_acc = history_df['val_acc'].max()
best_val_loss = history_df['val_loss'].min()

print(f"\n{'='*60}")
print(f"Mejor Train Acc:  {best_train_acc:.2f}%")
print(f"Mejor Val Acc:    {best_val_acc:.2f}%")
print(f"Mejor Val Loss:   {best_val_loss:.4f}")
print(f"{'='*60}")

---
## 🔟 Solución de Problemas <a id="10"></a>

### ❌ Error: Out of Memory (OOM)

**Soluciones:**
1. Reduce el batch size:
   ```python
   config['batch_size'] = 4  # o incluso 2
   ```
2. Limpia la memoria GPU (ejecuta la celda de "Limpiar Memoria GPU")
3. Reinicia el runtime: `Runtime > Restart runtime`

---

### ❌ Error: No GPU available

**Solución:**
1. Ve a `Runtime > Change runtime type`
2. Selecciona `GPU` como Hardware accelerator
3. Guarda y reconecta

---

### ⏱️ Sesión de Colab se Desconecta

**Información importante:**
- Colab gratuito: máximo 12 horas continuas
- Los checkpoints se guardan automáticamente cada N epochs
- Usa Google Drive para respaldos automáticos

**Para resumir entrenamiento:**
```python
# (Funcionalidad en desarrollo)
# config['resume_from'] = 'path/to/checkpoint.pt'
```

---

### 📁 Videos No Se Cargan

**Verificaciones:**
1. Verifica que los videos estén en la carpeta correcta
2. Ejecuta la celda de "Verificar Estructura de Datasets"
3. Revisa los paths en la configuración
4. Asegúrate que los videos sean `.mp4`

---

### 🐛 Otros Problemas

Si encuentras otros errores:
1. Revisa los logs en la consola
2. Verifica que todas las dependencias estén instaladas
3. Reinicia el runtime y vuelve a ejecutar desde el principio
4. Consulta la documentación en el repositorio GitHub

---

## ✅ ¡Experimento Completado!

### 📊 Resumen del Experimento

- **Dataset:** {DATASET_TYPE.upper()}
- **Versión:** {VERSION.upper()}
- **Clases:** {config['num_classes']}
- **Videos de Entrenamiento:** {len(train_dataset)}
- **Test Accuracy:** Ver resultados en la sección 6

### 📁 Archivos Generados

Todos los archivos están guardados en:
- **Google Drive:** `{DRIVE_ROOT}/`
- **Colab Local:** `{config['results_dir']}`

### 📚 Próximos Pasos

1. Analiza las visualizaciones generadas
2. Revisa el reporte TXT para tu tesis
3. Compara con otros experimentos (V1 vs V2)
4. Considera entrenar con el otro dataset (100 vs 300 clases)

---

**🎓 Desarrollado por:** Rafael Ovalle - UNAB

**📧 Contacto:** [tu-email@ejemplo.com]

**🔗 Repositorio:** https://github.com/Ov4llezz/AtiendeSenas-MVP

---

### ⭐ ¡Éxito con tu Tesis!